In [41]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import scipy.io as spio

Import Data from Matlab

In [42]:
mat = spio.loadmat('../ReflectionModel/dataset_2.mat')
features = mat['features']
labels = mat['labels']
print(len(features))
print(len(features[0]))

100000
744


Preprocess Data to make it 0-mean, 1 variance

In [43]:
feature_mean = np.mean(features,axis=0)
label_mean = np.mean(labels,axis=0)
label_std = np.std(labels,axis=0)
feature_std = np.std(features,axis=0)
print(feature_std.shape)
feature_mean_m = np.tile(feature_mean, [len(features),1])
feature_std_m = np.tile(feature_std, [len(features),1])
label_mean_m = np.tile(label_mean,[len(labels),1])
label_std_m = np.tile(label_std,[len(labels),1])
print(feature_std_m.shape)
features_normal = np.divide(features - feature_mean_m, feature_std_m)
labels_normal = np.divide(labels- label_mean_m, label_std_m)
print(features_normal.shape)
print(labels_normal.shape)
print(label_std)
print(feature_std)

(744,)
(100000, 744)
(100000, 744)
(100000, 2)
[ 5.75757782  4.33572888]
[ 1.50315946  1.50109093  1.4988298   1.49266982  1.48432403  1.47321853
  1.45962753  1.44957145  1.43547674  1.42550123  1.41776869  1.40891897
  1.40464574  1.40081011  1.39350686  1.3876605   1.38834796  1.38462617
  1.38282034  1.37929115  1.37467546  1.36807602  1.36257241  1.36075581
  1.35685574  1.35138349  1.34216772  1.33444303  1.3243377   1.31200297
  1.29890701  1.48734615  1.48479231  1.48126276  1.47927189  1.47873322
  1.47363626  1.47069967  1.47064985  1.47103399  1.46945672  1.46635492
  1.46368271  1.45621113  1.44749263  1.44078627  1.43670867  1.43340685
  1.4299623   1.4269948   1.42076747  1.41534622  1.40888781  1.40179645
  1.39467555  1.38957635  1.38892293  1.38715744  1.38159447  1.37412147
  1.36516209  1.35865435  1.86039562  1.84183167  1.8274101   1.81269607
  1.79934722  1.78957177  1.78205275  1.77188538  1.7611249   1.74892173
  1.73490111  1.72492656  1.7168491   1.70649949  1

In [44]:
train_pts = 60000
valid_pts = 20000
test_pts = 20000
print(type(features_normal[0][0]))
features_normal = features_normal.astype('float32')
print(type(features_normal[0][0]))
labels_normal = labels_normal.astype('float32')
train_dataset = (features_normal[0:train_pts ,:])
train_labels = labels_normal[0:train_pts,:]
valid_dataset = features_normal[train_pts:valid_pts+train_pts,:]
valid_labels = labels_normal[train_pts:valid_pts+train_pts,:]
test_dataset = features_normal[train_pts+valid_pts:train_pts+valid_pts+test_pts,:]
test_labels = labels_normal[train_pts+valid_pts:train_pts+valid_pts+test_pts,:]
print(train_dataset.shape)
print(train_dataset[0:10,0:10])

<type 'numpy.float64'>
<type 'numpy.float32'>
(60000, 744)
[[-0.58843237  1.04194999  1.55480695  0.34897208 -1.17734814 -1.47347283
  -0.15485772  1.35086918  1.43462837 -0.06087759]
 [ 0.5727374  -0.63181102  0.57865942 -0.48796439  0.29917082 -0.0819752
  -0.12569059  0.34876576 -0.49538198  0.58206981]
 [ 0.38792133 -0.09478956 -0.33191782  0.58606082 -0.70387477  0.53310448
  -0.23637578 -0.18864956  0.47202417 -0.68435276]
 [ 0.4830538  -0.41351202  0.27046362 -0.18620938  0.04199331  0.12849844
  -0.26418701  0.37733597 -0.46469939  0.56523752]
 [ 0.23897913  0.29706541 -0.21316612 -0.31462404  0.16421017  0.36679238
  -0.11064763 -0.33104613  0.09344901  0.36327749]
 [ 0.47181526 -0.51018226  0.46705303 -0.4285498   0.29008842 -0.14044641
   0.02227816  0.12073169 -0.25261331  0.38072529]
 [-0.39212355  0.21411426  0.09052867 -0.35841307  0.44956285 -0.35605821
   0.10936376  0.16798821 -0.39403951  0.50060964]
 [ 0.06375302  0.13484873 -0.30906481  0.44539991 -0.59131849  0.60

Setup TensorFlow graph

In [45]:
batch_size = 2048
num_hidden = [1024,512,256]
depth = 4
graph = tf.Graph()
keep_prob = 1
decay_rate = 0.5
decay_steps = 1500
num_features =features.shape[1]
num_labels = labels.shape[1]
print(num_features)
def get_logits_for_training(dataset,w,b):
    cur_mat = dataset
    for i in range(0,depth-1):
        cur_mat = tf.matmul(cur_mat,w[i])+b[i]
        cur_mat = tf.nn.dropout(tf.nn.relu(cur_mat),keep_prob)
    logits = tf.matmul(cur_mat,w[depth-1])+b[depth-1]
    return logits

def get_logits_for_prediction(dataset,w,b):
    cur_mat = dataset
    for i in range(0,depth-1):
        cur_mat = tf.matmul(cur_mat,w[i])+b[i]
        cur_mat = tf.nn.relu(cur_mat)
    logits = tf.matmul(cur_mat,w[depth-1])+b[depth-1]
    return logits
    
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, num_features))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w = []
  b = []
  w.append(tf.Variable(tf.truncated_normal([num_features, num_hidden[0]],stddev=0.002)))
  b.append(tf.zeros([num_hidden[0]]))
  for i in range(1,depth-1):
    w.append(tf.Variable(tf.truncated_normal([num_hidden[i-1],num_hidden[i]], stddev=0.002)))
    b.append(tf.zeros([num_hidden[i]]))
  w.append(tf.Variable(tf.truncated_normal([num_hidden[depth-2], num_labels],stddev=0.002)))
  b.append(tf.Variable(tf.zeros([num_labels])))
  
  # Training computation.  
  logits = get_logits_for_training(tf_train_dataset,w,b)
  #loss = tf.reduce_mean(
  #  tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.sqrt(tf.reduce_mean(tf.squared_difference(tf_train_labels, logits)))
  #Optimizer
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  logits_predict = get_logits_for_prediction(tf_train_dataset,w,b)
  train_prediction = (logits_predict)
  valid_prediction = (get_logits_for_prediction(tf_valid_dataset,w,b))
  test_prediction = (get_logits_for_prediction(tf_test_dataset,w,b))

744


In [46]:
def accuracy(predictions, labels):
  return np.sqrt(((predictions - labels) ** 2).mean())
num_steps = 10002
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions,l1 = session.run(
      [optimizer, loss, train_prediction,w], feed_dict=feed_dict)
    if (step % 100 == 1):
      print("Minibatch loss at step %d: %f" % (step, l))
      print(l1[1].shape)
        
      print("Minibatch accuracy: %.3f" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.3f" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.3f" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 1: 1.002859
(1024, 512)
Minibatch accuracy: 1.003
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 101: 0.993718
(1024, 512)
Minibatch accuracy: 0.994
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 201: 0.997682
(1024, 512)
Minibatch accuracy: 0.998
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 301: 0.995493
(1024, 512)
Minibatch accuracy: 0.995
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 401: 0.997666
(1024, 512)
Minibatch accuracy: 0.998
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 501: 0.994685
(1024, 512)
Minibatch accuracy: 0.995
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 601: 1.002511
(1024, 512)
Minibatch accuracy: 1.003
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 701: 0.998541
(1024, 512)
Minibatch accuracy: 0.999
Validation accuracy: 0.999
Test accuracy: 1.001
Miniba

Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 6701: 1.000623
(1024, 512)
Minibatch accuracy: 1.001
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 6801: 0.981924
(1024, 512)
Minibatch accuracy: 0.982
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 6901: 1.008277
(1024, 512)
Minibatch accuracy: 1.008
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 7001: 1.006326
(1024, 512)
Minibatch accuracy: 1.006
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 7101: 0.997971
(1024, 512)
Minibatch accuracy: 0.998
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 7201: 1.004239
(1024, 512)
Minibatch accuracy: 1.004
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 7301: 1.000283
(1024, 512)
Minibatch accuracy: 1.000
Validation accuracy: 0.999
Test accuracy: 1.001
Minibatch loss at step 7401: 1.001536
(1024, 512)
Minibatch accuracy: 1.002
Validati